In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!wget -q https://archive.apache.org/dist/spark/spark-2.2.3/spark-2.2.3-bin-hadoop2.7.tgz
!tar xf spark-2.2.3-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
!ls

gdrive	sample_data  spark-2.2.3-bin-hadoop2.7	spark-2.2.3-bin-hadoop2.7.tgz


In [0]:
import os       #importing os to set environment variable
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  !java -version       #check java version
install_java()

openjdk version "11.0.4" 2019-07-16
OpenJDK Runtime Environment (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3)
OpenJDK 64-Bit Server VM (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3, mixed mode, sharing)


In [0]:
!update-alternatives --config java

There are 2 choices for the alternative java (providing /usr/bin/java).

  Selection    Path                                            Priority   Status
------------------------------------------------------------
* 0            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      auto mode
  1            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      manual mode
  2            /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java   1081      manual mode

Press <enter> to keep the current choice[*], or type selection number: 2
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode


In [0]:
!java -version 

openjdk version "1.8.0_222"
OpenJDK Runtime Environment (build 1.8.0_222-8u222-b10-1ubuntu1~18.04.1-b10)
OpenJDK 64-Bit Server VM (build 25.222-b10, mixed mode)


In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.2.3-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()

In [0]:
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
conf = SparkConf().setAppName("MLlib")
sc = SparkContext(conf=conf)
sc.setLogLevel("WARN")
spark = SparkSession(sc)

In [334]:
payment_df = spark.read.format("csv").option("header", "true").load(r'/content/gdrive/My Drive/models/rental/payment.csv')
payment_df = payment_df.alias("payment_df")
payment_df.show(3)

+----------+-----------+--------+---------+------+-------------------+-------------------+
|payment_id|customer_id|staff_id|rental_id|amount|       payment_date|        last_update|
+----------+-----------+--------+---------+------+-------------------+-------------------+
|         1|          1|       1|       76|  2.99|2005-05-25 11:30:37|2006-02-15 22:12:30|
|         2|          1|       1|      573|  0.99|2005-05-28 10:35:23|2006-02-15 22:12:30|
|         3|          1|       1|     1185|  5.99|2005-06-15 00:54:12|2006-02-15 22:12:30|
+----------+-----------+--------+---------+------+-------------------+-------------------+
only showing top 3 rows



In [335]:
rental_df = spark.read.format("csv").option("header", "true").load(r'/content/gdrive/My Drive/models/rental/rental.csv')
rental_df = rental_df.alias('rental_df')
rental_df.show(3)

+---------+-------------------+------------+-----------+-------------------+--------+-------------------+
|rental_id|        rental_date|inventory_id|customer_id|        return_date|staff_id|        last_update|
+---------+-------------------+------------+-----------+-------------------+--------+-------------------+
|        1|2005-05-24 22:53:30|         367|        130|2005-05-26 22:04:30|       1|2006-02-15 21:30:53|
|        2|2005-05-24 22:54:33|        1525|        459|2005-05-28 19:40:33|       1|2006-02-15 21:30:53|
|        3|2005-05-24 23:03:39|        1711|        408|2005-06-01 22:12:39|       1|2006-02-15 21:30:53|
+---------+-------------------+------------+-----------+-------------------+--------+-------------------+
only showing top 3 rows



In [336]:
staff_df = spark.read.format("csv").option("header", "true").load(r'/content/gdrive/My Drive/models/rental/staff.csv')
staff_df = staff_df.alias('staff_df')
staff_df.show(3)

+--------+----------+---------+----------+-------+--------------------+--------+------+--------+--------------------+--------------+
|staff_id|first_name|last_name|address_id|picture|               email|store_id|active|username|            password|   last_update|
+--------+----------+---------+----------+-------+--------------------+--------+------+--------+--------------------+--------------+
|       1|      Mike|  Hillyer|         3|   NULL|Mike.Hillyer@saki...|       1|     1|    Mike|8cb2237d0679ca88d...|2/15/2006 3:57|
|       2|       Jon| Stephens|         4|   NULL|Jon.Stephens@saki...|       2|     1|     Jon|                NULL|2/15/2006 3:57|
+--------+----------+---------+----------+-------+--------------------+--------+------+--------+--------------------+--------------+



In [337]:
inventory_df = spark.read.format("csv").option("header", "true").load(r'/content/gdrive/My Drive/models/rental/inventory.csv')
inventory_df = inventory_df.alias('inventory_df')
inventory_df.show(3)

+------------+-------+--------+-------------------+
|inventory_id|film_id|store_id|        last_update|
+------------+-------+--------+-------------------+
|           1|      1|       1|2006-02-15 05:09:17|
|           2|      1|       1|2006-02-15 05:09:17|
|           3|      1|       1|2006-02-15 05:09:17|
+------------+-------+--------+-------------------+
only showing top 3 rows



In [338]:
store_df = spark.read.format("csv").option("header", "true").load(r'/content/gdrive/My Drive/models/rental/store.csv')
store_df = store_df.alias('store_df')
store_df.show(3)

+--------+----------------+----------+-------------------+
|store_id|manager_staff_id|address_id|        last_update|
+--------+----------------+----------+-------------------+
|       1|               1|         1|2006-02-15 04:57:12|
|       2|               2|         2|2006-02-15 04:57:12|
+--------+----------------+----------+-------------------+



In [339]:
address_df = spark.read.format("csv").option("header", "true").load(r'/content/gdrive/My Drive/models/rental/address.csv')
address_df = address_df.alias('address_df')
address_df.show(3)

+----------+------------------+--------+--------+-------+-----------+-----------+-------------------+
|address_id|           address|address2|district|city_id|postal_code|      phone|        last_update|
+----------+------------------+--------+--------+-------+-----------+-----------+-------------------+
|         1| 47 MySakila Drive|    NULL| Alberta|    300|       null|       null|2014-09-25 22:30:27|
|         2|28 MySQL Boulevard|    NULL|     QLD|    576|       null|       null|2014-09-25 22:30:09|
|         3| 23 Workhaven Lane|    NULL| Alberta|    300|       null|14033335568|2014-09-25 22:30:27|
+----------+------------------+--------+--------+-------+-----------+-----------+-------------------+
only showing top 3 rows



In [340]:
customer_df = spark.read.format("csv").option("header", "true").load(r'/content/gdrive/My Drive/models/rental/customer.csv')
customer_df = customer_df.alias('customer_df')
customer_df.show(3)

+-----------+--------+----------+---------+--------------------+----------+------+-------------------+-------------------+
|customer_id|store_id|first_name|last_name|               email|address_id|active|        create_date|        last_update|
+-----------+--------+----------+---------+--------------------+----------+------+-------------------+-------------------+
|          1|       1|      MARY|    SMITH|MARY.SMITH@sakila...|         5|     1|2006-02-14 22:04:36|2006-02-15 04:57:20|
|          2|       1|  PATRICIA|  JOHNSON|PATRICIA.JOHNSON@...|         6|     1|2006-02-14 22:04:36|2006-02-15 04:57:20|
|          3|       1|     LINDA| WILLIAMS|LINDA.WILLIAMS@sa...|         7|     1|2006-02-14 22:04:36|2006-02-15 04:57:20|
+-----------+--------+----------+---------+--------------------+----------+------+-------------------+-------------------+
only showing top 3 rows



In [341]:
film_df = spark.read.format("csv").option("header", "true").load(r'/content/gdrive/My Drive/models/rental/film.csv')
film_df = film_df.alias('film_df')
film_df.show(3)

+-------+----------------+--------------------+------------+-----------+--------------------+---------------+-----------+------+----------------+------+--------------------+-------------------+
|film_id|           title|         description|release_year|language_id|original_language_id|rental_duration|rental_rate|length|replacement_cost|rating|    special_features|        last_update|
+-------+----------------+--------------------+------------+-----------+--------------------+---------------+-----------+------+----------------+------+--------------------+-------------------+
|      1|ACADEMY DINOSAUR|A Epic Drama of a...|        2006|          1|                NULL|              6|       0.99|    86|           20.99|    PG|Deleted Scenes,Be...|2006-02-15 05:03:42|
|      2|  ACE GOLDFINGER|A Astounding Epis...|        2006|          1|                NULL|              3|       4.99|    48|           12.99|     G|Trailers,Deleted ...|2006-02-15 05:03:42|
|      3|ADAPTATION HOLES|A As

In [344]:
print("Homework solution start here follow by question order")

Homework solution start here follow by question order


In [342]:
payment_df.withColumn("amount", payment_df["amount"].cast("double"))
import pyspark.sql.functions as func
#payment_df.groupby('customer_id').agg(func.sum(payment_df.amount).alias('total_amount')).orderBy('total_amount',ascending = False).show(3)
payment_df.createOrReplaceTempView("payment_df")
spark.sql("select customer_id, sum(amount) as total_amount from payment_df group by customer_id order by total_amount DESC").show(3)

+-----------+------------------+
|customer_id|      total_amount|
+-----------+------------------+
|        526| 221.5500000000001|
|        148| 216.5400000000001|
|        144|195.58000000000007|
+-----------+------------------+
only showing top 3 rows



In [250]:
rental_df.createOrReplaceTempView("rental_df")
spark.sql("select * from rental_df where year(rental_date) = 2005 order by return_date").show(3)

+---------+-------------------+------------+-----------+-------------------+--------+-------------------+
|rental_id|        rental_date|inventory_id|customer_id|        return_date|staff_id|        last_update|
+---------+-------------------+------------+-----------+-------------------+--------+-------------------+
|       32|2005-05-25 04:06:21|        3832|        230|2005-05-25 23:55:21|       1|2006-02-15 21:30:53|
|       21|2005-05-25 01:59:46|         146|        388|2005-05-26 01:01:46|       2|2006-02-15 21:30:53|
|       14|2005-05-25 00:31:15|        2701|        446|2005-05-26 02:56:15|       1|2006-02-15 21:30:53|
+---------+-------------------+------------+-----------+-------------------+--------+-------------------+
only showing top 3 rows



In [185]:
spark.sql("select staff_id, count(rental_id) as count from rental_df group by staff_id").show()

+--------+-----+
|staff_id|count|
+--------+-----+
|       1| 8040|
|       2| 8004|
+--------+-----+



In [186]:
staff_df.createOrReplaceTempView("staff_df")
spark.sql("select first(first_name) as first_name, first(last_name) as last_name, count(rental_id) as count from staff_df inner join rental_df on rental_df.staff_id = staff_df.staff_id group by staff_df.staff_id").show()

+----------+---------+-----+
|first_name|last_name|count|
+----------+---------+-----+
|      Mike|  Hillyer| 8040|
|       Jon| Stephens| 8004|
+----------+---------+-----+



In [382]:
inventory_df.createOrReplaceTempView("inventory_df")
store_df.createOrReplaceTempView("store_df")
address_df.createOrReplaceTempView("address_df")
customer_df.createOrReplaceTempView("customer_df")
film_df.createOrReplaceTempView("film_df")
spark.sql("""select r.rental_date, r.return_date, a.address, f.title, f.description, c.first_name, c.last_name, s.first_name, s.last_name
from rental_df as r
join inventory_df as i
on r.inventory_id = i.inventory_id
join store_df as st
on i.store_id = st.store_id
join address_df as a
on st.address_id = a.address_id
join customer_df as c
on r.customer_id = c.customer_id
join staff_df as s
on s.store_id = c.store_id
join film_df as f
on i.film_id = f.film_id
order by r.rental_date
 """).show(3)

+-------------------+-------------------+------------------+---------------+--------------------+----------+---------+----------+---------+
|        rental_date|        return_date|           address|          title|         description|first_name|last_name|first_name|last_name|
+-------------------+-------------------+------------------+---------------+--------------------+----------+---------+----------+---------+
|2005-05-24 22:53:30|2005-05-26 22:04:30| 47 MySakila Drive|BLANKET BEVERLY|A Emotional Docum...| CHARLOTTE|   HUNTER|      Mike|  Hillyer|
|2005-05-24 22:54:33|2005-05-28 19:40:33|28 MySQL Boulevard|   FREAKY POCUS|A Fast-Paced Docu...|     TOMMY|  COLLAZO|      Mike|  Hillyer|
|2005-05-24 23:03:39|2005-06-01 22:12:39|28 MySQL Boulevard|  GRADUATE LORD|A Lacklusture Epi...|    MANUEL|  MURRELL|      Mike|  Hillyer|
+-------------------+-------------------+------------------+---------------+--------------------+----------+---------+----------+---------+
only showing top 3 r

In [263]:
spark.sql("""select customer_id, total_amount from
(select customer_id, sum(amount) as total_amount from payment_df where year(payment_date) =2005 group by customer_id)
where total_amount > 200""").show()

+-----------+-----------------+
|customer_id|     total_amount|
+-----------+-----------------+
|        526|221.5500000000001|
|        148|216.5400000000001|
+-----------+-----------------+



In [299]:
customer_df.createOrReplaceTempView("customer_df")
spark.sql("""select * from
(select first(customer_df.first_name) as first_name, first(customer_df.last_name) as last_name, sum(amount) as total_amount from payment_df
inner join customer_df
on customer_df.customer_id = payment_df.customer_id where year(payment_date) =2005 group by payment_df.customer_id)
where total_amount > 200 """).show()

+----------+---------+-----------------+
|first_name|last_name|     total_amount|
+----------+---------+-----------------+
|      KARL|     SEAL|221.5500000000001|
|   ELEANOR|     HUNT|216.5400000000001|
+----------+---------+-----------------+



In [333]:
c2005 = spark.sql("select customer_id, year(rental_date) as rentyear from rental_df where year(rental_date) = 2005")
c2006 = spark.sql("select customer_id, year(rental_date) as rentyear from rental_df where year(rental_date) = 2006")
c2005.createOrReplaceTempView("c2005")
c2006.createOrReplaceTempView("c2006")
spark.sql("""select distinct(c2005.customer_id) as customer_id_in_2005_not_in_2006, c2005.rentyear from c2005
where c2005.customer_id not in (
    select c2006.customer_id from c2006
) """).show(10)
spark.sql("""select count(distinct(c2005.customer_id)) as customer_id_in_2005_not_in_2006 from c2005
where c2005.customer_id not in (
    select c2006.customer_id from c2006
) """).show()
spark.sql("""select distinct(c2006.customer_id) as customer_id_in_2006_not_in_2005, c2006.rentyear from c2006
where c2006.customer_id not in (
    select c2005.customer_id from c2005
) """).show(10)
spark.sql("""select count(c2006.customer_id) as customer_id_in_2006_not_in_2005 from c2006
where c2006.customer_id not in (
    select c2005.customer_id from c2005
) """).show(10)

+-------------------------------+--------+
|customer_id_in_2005_not_in_2006|rentyear|
+-------------------------------+--------+
|                            413|    2005|
|                            377|    2005|
|                            406|    2005|
|                            148|    2005|
|                            463|    2005|
|                             26|    2005|
|                            358|    2005|
|                             62|    2005|
|                            285|    2005|
|                            392|    2005|
+-------------------------------+--------+
only showing top 10 rows

+-------------------------------+
|customer_id_in_2005_not_in_2006|
+-------------------------------+
|                            441|
+-------------------------------+

+-------------------------------+--------+
|customer_id_in_2006_not_in_2005|rentyear|
+-------------------------------+--------+
+-------------------------------+--------+

+---------------------------